# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
#Array of non-numeric columns in dataframe
print(np.where(df.dtypes!='float64'))

(array([ 3,  5,  6,  7,  8, 16, 27, 30, 84, 85]),)


In [6]:
#split issue_d column into month and year
new_column = df['issue_d'].str.split('-',n=1,expand=True)
df['month_issue_d'] = new_column[0]
df['year_issue_d'] = new_column[1]
df.drop(columns=['issue_d'],inplace=True)
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,month_issue_d,year_issue_d
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,Mar,2019
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,Mar,2019
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,Mar,2019
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,Mar,2019
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,Mar,2019


In [7]:
# create label encoder object
le = LabelEncoder()

# Fitting and encoding appropriate columns with the LabelEncoder
df['home_ownership'] = le.fit_transform(df['home_ownership'])
df['verification_status'] = le.fit_transform(df['verification_status'])
df['loan_status'] = le.fit_transform(df['loan_status'])
df['pymnt_plan'] = le.fit_transform(df['pymnt_plan'])
df['initial_list_status'] = le.fit_transform(df['initial_list_status'])
df['next_pymnt_d'] = le.fit_transform(df['next_pymnt_d'])
df['application_type'] = le.fit_transform(df['application_type'])
df['hardship_flag'] = le.fit_transform(df['hardship_flag'])
df['debt_settlement_flag'] = le.fit_transform(df['debt_settlement_flag'])
df['month_issue_d'] = le.fit_transform(df['month_issue_d'])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,month_issue_d,year_issue_d
0,10500.0,0.1719,375.35,3,66000.0,1,1,0,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,2,2019
1,25000.0,0.2000,929.09,1,105000.0,2,1,0,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0,2,2019
2,20000.0,0.2000,529.88,1,56000.0,2,1,0,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0,2,2019
3,10000.0,0.1640,353.55,3,92000.0,2,1,0,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,2,2019
4,22000.0,0.1474,520.39,1,52000.0,0,1,0,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0,2,2019


In [8]:
#convert year column to int
df['year_issue_d'] = df['year_issue_d'].astype(str).astype(int)

df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                  int64
annual_inc                    float64
                               ...   
total_il_high_credit_limit    float64
hardship_flag                   int64
debt_settlement_flag            int64
month_issue_d                   int64
year_issue_d                    int64
Length: 87, dtype: object

## Split the Data into Training and Testing

In [9]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,month_issue_d,year_issue_d
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,0.805542,2019.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.714932,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,0.000000,2019.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,0.000000,2019.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,1.000000,2019.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,1.000000,2019.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,2.000000,2019.0


In [11]:
# Check the balance of our target values
y_value, y_count = np.unique(y,return_counts=True)
print(np.column_stack((y_value,y_count)))

[[    0   347]
 [    1 68470]]


In [12]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [15]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit the model
brf_model = brf_model.fit(X_train_scaled, y_train)

# Make predictions
brf_predictions = brf_model.predict(X_test_scaled)

In [17]:
# Calculated the balanced accuracy score
brf_accuracy = balanced_accuracy_score(y_test, brf_predictions)
print(f"BRF Balanced Accuracy Score : {brf_accuracy}")

BRF Balanced Accuracy Score : 0.7335852767924127


In [18]:
# Display the confusion matrix
# Calculating the confusion matrix
brf_cm = confusion_matrix(y_test, brf_predictions)
brf_cm_df = pd.DataFrame(
    brf_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

print("BalancedRandomForest Confusion Matrix")
display(brf_cm_df)

BalancedRandomForest Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,62,39
Actual 1,2509,14595


In [19]:
# Print the imbalanced classification report
print("BRF Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, brf_predictions))

BRF Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.61      0.85      0.05      0.72      0.51       101
          1       1.00      0.85      0.61      0.92      0.72      0.54     17104

avg / total       0.99      0.85      0.62      0.91      0.72      0.54     17205



In [20]:
# List the features sorted in descending order by feature importance
importance_sorted = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
importance_sorted[:30]

[(0.0768342915643905, 'total_rec_prncp'),
 (0.05686504705456714, 'total_rec_int'),
 (0.05536255116598874, 'last_pymnt_amnt'),
 (0.05407538543939351, 'total_pymnt_inv'),
 (0.051862633369057824, 'total_pymnt'),
 (0.0323709900280044, 'int_rate'),
 (0.022391803746568545, 'dti'),
 (0.019548055985890725, 'annual_inc'),
 (0.018985833303554005, 'mths_since_recent_inq'),
 (0.018350007824699047, 'month_issue_d'),
 (0.01742384974698456, 'avg_cur_bal'),
 (0.01667314302875787, 'tot_hi_cred_lim'),
 (0.01613700503051451, 'installment'),
 (0.015974194270878782, 'total_rev_hi_lim'),
 (0.015767198003146923, 'mths_since_rcnt_il'),
 (0.01569079353083912, 'bc_open_to_buy'),
 (0.015631562585443095, 'all_util'),
 (0.015141462607020626, 'total_bc_limit'),
 (0.015136290668932881, 'mo_sin_old_il_acct'),
 (0.015071036020410131, 'max_bal_bc'),
 (0.014836704985337299, 'mo_sin_old_rev_tl_op'),
 (0.014152764987783846, 'bc_util'),
 (0.013796918254431276, 'tot_cur_bal'),
 (0.013741883309982862, 'total_bal_ex_mort'),
 

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit the model
eec_model = eec_model.fit(X_train_scaled, y_train)

# Make predictions
eec_predictions = eec_model.predict(X_test_scaled)

In [22]:
# Calculated the balanced accuracy score
eec_accuracy = balanced_accuracy_score(y_test, eec_predictions)
print(f"EEC Balanced Accuracy Score : {eec_accuracy}")

EEC Balanced Accuracy Score : 0.931601605553446


In [23]:
# Display the confusion matrix
# Calculating the confusion matrix
eec_cm = confusion_matrix(y_test, eec_predictions)
eec_cm_df = pd.DataFrame(
    eec_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

print("EasyEnsembleClassifier Confusion Matrix")
display(eec_cm_df)

EasyEnsembleClassifier Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,93,8
Actual 1,985,16119


In [24]:
# Print the imbalanced classification report
print("EEC Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, eec_predictions))

EEC Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier with a score of 0.9028, was better in this regard.

2. Which model had the best recall score?

    Easy Ensemble Classifier with a score of 0.99, was better in this regard.

3. Which model had the best geometric mean score?

    Easy Ensemble Classifier with a score of 0.90, was better in this regard.

4. What are the top three features?
     
    Below are the the top three features and their scores:
    
    - (0.0813986646925866, 'total_pymnt_inv')
    - (0.06524574330478597, 'total_rec_prncp')
    - (0.06273809567243768, 'total_rec_int')